In [ ]:
import os
import json
import torch

import torch.utils.data as data
import torchvision.transforms.v2 as tfs

from PIL import Image

In [ ]:
class MyDatasets(data.Dataset):
    def __init__(self, path, train=True, transform=None):
        self.path = os.path.join(path, "train" if train else "test")
        self.transform = transform

        with open(os.path.join(path, "format.json"), "r") as fp:
            self.format = json.load(fp)

        self.length = 0
        self.files = []
        self.targets = torch.eye(10) # единичная матрица

        for _dir, _target in self.format.items():
            path = os.path.join(self.path, _dir)
            list_files = os.listdir(path)
            self.length += len(list_files)
            self.files.extend(map(lambda _x: (os.path.join(path, _x), _target), list_files))


    def __getitem__(self, item):
        path_file, target = self.files[item]
        tensor_target = self.targets[target] # из единичной матрицы нужная строка
        img = Image.open(path_file)

        if self.transform:
            img = self.transform(img).ravel().float() / 255.0

        return img, tensor_target # входные, выходные


    def __len__(self):
        return self.length
